# Prepare data from raw zip (only once)

In [1]:
import pandas as pd
from zipfile import ZipFile
import shutil
from os import listdir, rename, makedirs
from tqdm.notebook import tqdm

In [1]:
# loading the temp.zip and creating a zip object
with ZipFile('data/hot_or_not_image_and_rating_data.zip', 'r') as zObject:
    # Extracting all the members of the zip
    zObject.extractall(path='data')

In [10]:
makedirs('data/images', exist_ok=True)
makedirs('data/texts', exist_ok=True)
path_dir = 'data'
for gender in ('female', 'male'):
    for file in tqdm(listdir(f'{path_dir}/{gender}')):
        if file[-3:] == 'jpg':
            file_type = 'images'
        elif file[-3:] == 'txt':
            file_type = 'texts'
        else:
            continue
        shutil.copy(f'{path_dir}/{gender}/{file}', f'{path_dir}/{file_type}/{file}')
        rename(f'{path_dir}/{file_type}/{file}', f'{path_dir}/{file_type}/{gender[0]}_{file}')

  0%|          | 0/2001 [00:00<?, ?it/s]

  0%|          | 0/2001 [00:00<?, ?it/s]

# Create and save Dataframe with biography information

In [6]:
ratings = []
n_ratings = []
texts = []
ids = []
for file in listdir('data/texts'):
    ids.append(file[:-4])
    with open(f'data/texts/{file}', 'r') as f:
        ratings.append(float(f.readline()))
        n_ratings.append(int(f.readline()))
        texts.append(f.readlines()[-1])

In [7]:
df = pd.DataFrame(data={'id': ids, 'number_ratings': n_ratings, 'avg_rating': ratings, 'text': texts})

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   object 
 1   number_ratings  2000 non-null   int64  
 2   avg_rating      2000 non-null   float64
 3   text            2000 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 62.6+ KB


In [9]:
df.to_parquet('data/text_data.parquet.gzip')